In [1]:
import sys
# import tk for getting the directory faster. dont need this in a command line/server version
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import numpy as np
import bigfish.detection 
import bigfish.stack
import bigfish.plot
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import random
import math
import json
# if you dont need to plot in jupyter you don need these. Some magic interperters need to be removed for command line version. 
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Glob and tifffile are needed
from glob import glob
from tifffile import imread,imwrite
# csb deep is to take normalization 
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
# This is your stardist models and everything in stardist coming from. 
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D
from skimage import segmentation
import bigfish.multistack as multistack
# Set random seed for you color map. You do not really need this to be 6 all the time, but its okay. 
np.random.seed(6)
lbl_cmap = random_label_cmap()

In [5]:
controlImage = False 
custom_model = True
modelDirectory = "/Users/eliasguan/Desktop/models"
controlPath = None
smFISHChannelPath = "/Users/eliasguan/Desktop/Lab Meeting Presentation and Progress/Report To Chris 103024/Example Image/Original_smFISH.tif"
nucleiChannelPath = "/Users/eliasguan/Desktop/EG_1112_WntP2/Image5/405/Region5_405.tif"
nuclei_projection_size = 10
# Set your thresh hold for stardist segmentation 
detection_thresh = (0.4,0.3)
# If the image is normalized (i.e normalized from CSBdeep or other method, use True, or use False)
normalizedImage = False
# Change your normalize threshold here. If it is True previously, you dont need to change it. 
normalize_thresh = (0.1, 99.9)
# Run all then you are good.
# Do you want to use a custo trained model here. 
# Here if you need to have a custom model the system will ask you for a custom model directory
custom_model = True
# Here select your random sample percentage in z-stack for nuclei segmentation estimation. 0 -100 is the percentage range, 100% means each z-stack will be selected and sampled. 
samplePercentage = 100
# Here select your random sample region size for nuclei segmentation estimation. 500X5000 means 500 X 500 pixel in each z-stack will be selected
sampleRegionSize = (500,500)
# Here set the nuclei label expanding radius
labelExpansionSize = 20

In [6]:
# Here I will put most of the varaibles need for running python 
# File path: This will include: A control channel for checking intensities and other parameters
# If there is no control channel, please change controlImage = False
# smFISHChannelPath for smFISH channel, the signal you want to segement in this case 
# nucleiChannelPath for the nuclei channel locations 
# assuming you are running 3D detection. If not please develop a separate code for this. 
controlImage = False 
custom_model = True
modelDirectory = "/Users/eliasguan/Desktop/models"
controlPath = None
smFISHChannelPath = "/Users/eliasguan/Desktop/EG_1112_WntP2/Image5/565/Region5_565.tif"
nucleiChannelPath = "/Users/eliasguan/Desktop/EG_1112_WntP2/Image5/405/Region5_405.tif"
# Set Parameters for detection. Here minimal distance is the minimal distance between spots. 
# Note this will be consistent for both control and the smFISH channel. 
# Unless specified separately, all these three number tuples are z,y,x in order. 
minimal_distance = (2,2,2)
# Set the Gaussian LoG filter Kernel size. Recommend to start with 1,1.5,1.5 and increase if you need more. 
# I don`t think you need this different from control and experimental image. 
kernel_size = (1,1.5,1.5)
# Set the voxel size. This is determined by the pixel size of your microscope. Please contact microscopt manufactuer and convert resolution to voxel size. 
# unit is nm, please change to nm and note this should be the same for control and your experimental image. 
# I specifically allow this code to run different voxel size for control and experimental image, but for a good experiment you should not do it like that. 
control_voxel_size = (361,75,75)
voxel_size = (361,75,75)
# Set the spot size as your expected spot size 
spot_size = (600, 300, 300)
decomposition_thresh = (0.7,1,5)
# Recommend start with 4 in planarian. You can have more. Recommend turn spotsRadiusDetection = True and run the radius test
# Usually the largest radius/the average radius is what you want. 
min_spots_for_clusters = 4
# Enter the radius for spot for detecting clusters. 
# If need refer to the spotsRadiusDetection to set a good reference
radius_for_spots = 250 
# Here you need to define the spot plotting size You can make it as large as possible. 
plot_spot_size = 4
# Here you add the nuclei projection size 

In [24]:
smFISHChannelArray = imread(smFISHChannelPath)
# Get your log filter and write it in the results folder 
smFISHChannelArray_LoG = bigfish.stack.log_filter(smFISHChannelArray, kernel_size)
imwrite('smFISHChannelArray_LoG.tif', smFISHChannelArray_LoG, photometric='minisblack')
# Get detection of your spots 
smFISH_spots_zero, smFISH_threshold= bigfish.detection.detect_spots(
        images=smFISHChannelArray,
        threshold = 0,
        return_threshold=True,
        voxel_size= voxel_size,  # in nanometer (one value per dimension zyx)
        spot_radius= spot_size, # in nanometer (one value per dimension zyx)
        log_kernel_size= kernel_size,
        minimum_distance= minimal_distance)


In [25]:
def generate_coordinates_2D(y, x, shape, iteration =4, get_inner_spot = False):
    # Initial collection with the starting coordinate (x, y)
    coordinates_collection = [(y, x)]
    max_y = (shape[0]-1)
    max_x = (shape[1]-1)

    for _ in range(iteration):
        # Copy the current collection to avoid modifying it while iterating
        current_coordinates = coordinates_collection.copy()

        # Loop through each coordinate in the current collection
        for coord in current_coordinates:
            # Extract x and y values from the coordinate
            cy, cx = coord

            # Add neighboring coordinates to the collection if within bounds
            if cx + 1 <= max_x:
                coordinates_collection.append((cy,cx+1))
            if 0 <= cx - 1:
                coordinates_collection.append((cy,cx-1))
            if cy + 1 < max_y:
                coordinates_collection.append((cy+1, cx))
            if 0 <= cy - 1:
                coordinates_collection.append((cy-1, cx))

    # Remove duplicates by converting the list to a set and then back to a list
    coordinates_collection = list(set(coordinates_collection))
    if get_inner_spot:
        return coordinates_collection
    else:
        coordinates_collection = [coord for coord in coordinates_collection if abs(coord[0] - y) + abs(coord[1] - x) == iteration]
        return coordinates_collection

In [27]:
spotPlot = np.zeros(smFISHChannelArray.shape,dtype=np.uint8)
for i in tqdm(range(len(smFISH_spots_zero))):
    shape = [spotPlot.shape[1],spotPlot.shape[2]]
    y = smFISH_spots_zero[i][1]
    x = smFISH_spots_zero[i][2]
    z = smFISH_spots_zero[i][0]
    plot_location = generate_coordinates_2D(y,x, shape, iteration = 4 , get_inner_spot = False)
    for item in plot_location:
        spotPlot[z,item[0],item[1]] = 255
imwrite('spotPlot_zero.tif',spotPlot, photometric = 'minisblack')

100%|██████████████████████████████| 1546308/1546308 [02:31<00:00, 10237.63it/s]


In [23]:
smFISH_spots_zero[1]

0

In [29]:
import os
os.getcwd()

'/Users/eliasguan/Desktop/Jupyter_Notebooks'

In [63]:
def find_spots_around_example(coordinate, array, max_iterations = 10):
    """
    Find all spots around the given coordinate in a 3D array until reaching a value of 0.
    input: coordinate: the coordinate of the spot 
           array: The image array, please us the LoG filtered image
    Output: a List of all array found around the coordinate 
    """
    
    # Start with outer_edge only containing this one single point
    outer_edge = [np.array(coordinate,dtype = np.uint16)]
    export_outer_edge = []
    # Start with the spot collection only containing this one single point
    spots_collection = [np.array(coordinate,dtype = np.uint16)]
    # Set up array shape 
    shape = array.shape
    # Go through max iteration numbers 
    counter = 0
    for i in range(max_iterations):
        # Set up new outer edge at the start, and replace the old outer edge with this on the next round 
        new_outer_edge = []
        flag = False 
        blank_image = np.zeros(array.shape,dtype=np.uint8)
        for spot in spots_collection:
            blank_image[spot[0],spot[1],spot[2]] = 255
        imwrite('spotPlot'+str(counter)+".tif", blank_image, photometric = 'minisblack')
        counter = counter+1
        # Iterate through all items in outer_edge
        for item in outer_edge:
            # Keep the length of the current outeredge
            # Fist see if coordinate in this item is greater than 1, if it is 0 you can not iterate to the left.
            if item[2]>=1:
                # if possible assign left 
                left = np.array([item[0],item[1],item[2]-1],dtype = np.int16)
                # Check if the point in the image is greater than 0, if it is 0 it will not be in the spot collection
                # Here I have three tests: First, if the spot is alreay in the collection. This avoids backsearching 
                # Second, if the spot is greater than 0 This means you reach an edge then you should immediately stop searching in this direction
                # Third, if the spot is greater than the original spot. This avoids accidentally meeting a second spot in the same area. 
                # If all these checks are met, add the spot to the left into both outer edge, and the spot collection. 
                if sum(np.array_equal(point_outer_edge, left) for point_outer_edge in spots_collection) == 0 and array[left[0],left[1],left[2]] > 0 and array[item[0],item[1],item[2]] >= array[left[0],left[1],left[2]]: 
                    # If this pass we add this point to the new outer edge 
                    # If this pass we add this point to the spots collection 
                    new_outer_edge.append(left)
                    spots_collection.append(left)
            # Then see if coordinate in this item is to the right edge of the image. If it is this case you can not iterate to the right
            if item[2]+1 <= shape[2]-1:
                right = np.array([item[0],item[1],item[2]+1],dtype = np.uint16)
                if sum(np.array_equal(point_outer_edge, right) for point_outer_edge in spots_collection) == 0 and array[right[0],right[1],right[2]] > 0 and array[item[0],item[1],item[2]] >= array[right[0],right[1],right[2]]:
                    # If this pass we add this point to the new outer edge 
                    # If this pass we add this point to the spots collection 
                    new_outer_edge.append(right)
                    spots_collection.append(right)
            # Do exact same thing for Up and down: 
            if item[1]>=1:
                up = np.array([item[0],item[1]-1,item[2]],dtype = np.uint16)
                if sum(np.array_equal(point_outer_edge, up) for point_outer_edge in spots_collection) == 0 and array[up[0],up[1],up[2]] > 0 and array[item[0],item[1],item[2]] >= array[up[0],up[1],up[2]]:
                    new_outer_edge.append(up)
                    spots_collection.append(up)
            if item[1]+1 <=shape[1]-1:
                down = np.array([item[0],item[1]+1,item[2]],dtype = np.uint16)
                if sum(np.array_equal(point_outer_edge, down) for point_outer_edge in spots_collection) == 0 and array[down[0],down[1],down[2]] > 0 and array[item[0],item[1],item[2]] >= array[down[0],down[1],down[2]]:
                    new_outer_edge.append(down)
                    spots_collection.append(down)
            # Do exact the same thing for top and bottom 
            if item[0]>=1:
                top = np.array([item[0]-1,item[1],item[2]],dtype = np.uint16)
                if sum(np.array_equal(point_outer_edge, top) for point_outer_edge in spots_collection) == 0 and array[top[0],top[1],top[2]] > 0 and array[item[0],item[1],item[2]] >= array[top[0],top[1],top[2]]:
                    new_outer_edge.append(top)
                    spots_collection.append(top)
            if item[0]+1 <= shape[0]-1:
                bottom = np.array([item[0]+1,item[1],item[2]],dtype = np.uint16)
                if sum(np.array_equal(point_outer_edge, bottom) for point_outer_edge in spots_collection) == 0 and array[bottom[0],bottom[1],bottom[2]] > 0 and array[item[0],item[1],item[2]] >= array[bottom[0],bottom[1],bottom[2]]:
                    new_outer_edge.append(bottom)
                    spots_collection.append(bottom)
            # Check if your new outer edge is empty. In this case you can not expand anyfurther so just break it. 
            if len(new_outer_edge) == 0: 
                flag = True
                break 
            # Here it is the end of inner loop

        # If flag is changed break the outloop as well       
        if flag == True: 
            break 
        # If you have other points in the new outer edge then you need to replace the old outer edge with the new outer edge. 
        else:
            outer_edge = new_outer_edge
        
    # Export your results 
    return np.array(spots_collection)

In [39]:
example_path = '/Users/eliasguan/Desktop/Lab Meeting Presentation and Progress/Example_Single_Spot/single_spot_exampletif.tif'
os.chdir(os.path.dirname(example_path))

In [60]:
smFISHChannelArray = imread(example_path)
# Get your log filter and write it in the results folder 
smFISHChannelArray_LoG = bigfish.stack.log_filter(smFISHChannelArray, kernel_size)
imwrite('smFISHChannelArray_LoG.tif', smFISHChannelArray_LoG, photometric='minisblack')
# Get detection of your spots 
smFISH_spots_zero, smFISH_threshold= bigfish.detection.detect_spots(
        images=smFISHChannelArray,
        return_threshold=True,
        voxel_size= voxel_size,  # in nanometer (one value per dimension zyx)
        spot_radius= spot_size, # in nanometer (one value per dimension zyx)
        log_kernel_size= kernel_size,
        minimum_distance= minimal_distance)

In [61]:
spot_center= [3,8,9]

In [69]:
a = find_spots_around_example(spot_center, smFISHChannelArray_LoG, max_iterations = 10)

In [70]:
os.getcwd()

'/Users/eliasguan/Desktop/Lab Meeting Presentation and Progress/Example_Single_Spot'

In [71]:
blank_image =  np.zeros(smFISHChannelArray_LoG.shape,dtype=np.uint8)
for spot in a:
    
    blank_image[spot[0],spot[1],spot[2]] = 255
imwrite('spotPlot'+str(6)+".tif", blank_image, photometric = 'minisblack')